<a href="https://colab.research.google.com/github/XiuqiXi/colabNotebook/blob/main/ManimExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

In [ ]:
from manim import *

In [38]:
%%manim -qm -v WARNING SquareToCircle

class SquareToCircle(Scene):
   def construct(self):
      square = Square()
      circle = Circle()
      circle.set_fill(RED, opacity=0.9)
      self.play(Create(circle))
      self.play(Transform(circle, square))
      self.wait()

In [39]:
%%manim -qm -v WARNING SquareAndCircle

class SquareAndCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        circle.set_fill(PINK, opacity=0.5)  # set the color and transparency

        square = Square()  # create a square
        square.set_fill(BLUE, opacity=0.5)  # set the color and transparency

        square.next_to(circle, UP, buff=-1)  # set the position
        self.play(Create(circle), Create(square))  # show the shapes on screen

In [40]:
%%manim -qm -v WARNING MathTeXDemo

class MathTeXDemo(Scene):
    def construct(self):
        rtarrow0 = MathTex(r"B_{R}=\frac{(1-\chi) \Delta H Y_{\infty} s-c_{p g}\left(T_{i g}-T_{\infty}\right)}{L+\left(\dot{q}_{f, r}^{\prime \prime}-\dot{q}_{s, r}^{\prime \prime}-\dot{q}_{s, c}^{\prime \prime}\right) / \dot{m}_{\mathrm{f}}^{\prime \prime}}", font_size=40)
        rtarrow1 = Tex(r"$B=\frac{\Delta H Y_{\infty \infty} s-c_{p g}\left(T_{\text {ig }}-T_{\infty}\right)}{L+Q}$", font_size=40)

        rtarrow1.next_to(rtarrow0, UP, buff=1.5)  # set the position

        self.play(Create(rtarrow0), Create(rtarrow1))

        self.play(Transform(rtarrow1, rtarrow0))

        self.wait()


In [50]:
import numpy as np

# Original Function
func_period_from, func_period_to = -PI, PI
func_discontinuities = np.arange(-PI * 4, PI * 4, PI)


def func(x):
    return 0 if x >= 0 else 1


# Fourier series for the func above
def series(x, num_terms):
    sum_terms = 1 / 2

    # reverse order
    # adding up from small numbers
    for n in range(num_terms, 0, -1):
        if (n % 2) == 0:
            continue
        sum_terms += -(2 / (n * PI)) * np.sin(n * x)

    return sum_terms


# Tex
# Web Tool : Hand writing math expression to Tex
#   https://webdemo.myscript.com/views/math/index.html#
tex_func = r"""
f(x) =
\begin{cases}
    \hspace{.4em}
    1 \quad ( \, -\pi \leq x < 0 \, ) \\
    \hspace{.4em}
    0 \quad ( \, 0 \leq x < \pi \, )
\end{cases}
"""
tex_sim_or_equals = r"\sim"
tex_series = r"""
\dfrac{1}{2} +
\sum_{n=1}^{\infty}
    \dfrac{1}{n\pi}
    \Big\{ (-1) ^{n}-1 \Big\} \sin nx
"""

# Number of terms
series_start, series_end, series_step = 1, 19, 2
series_run_time = 15

series_more_start, series_more_end, series_more_step = 21, 201, 10
series_more_run_time = 10

# Axes
x_min, x_max, x_step = -3.4 * PI, 3.4 * PI, PI
y_min, y_max, y_step = -0.1, 1.5, 1
y_length = 3
include_ticks = False
include_tip = False
is_pi_label = True
show_x_0_label = True

# Function plot
func_x_range_min, func_x_range_max = -3.3 * PI, 3.3 * PI
plot_x_delta = 5e-3

# Color
COLOR_MESSAGE = WHITE
COLOR_FUNC = GREEN
COLOR_PERIODIC = YELLOW
COLOR_SERIES = BLUE
COLOR_TILDE = WHITE
COLOR_N_TERM_COUNT = WHITE

# Message displayed before the formula
message = "Fourier Series for"

In [51]:
%%manim -qm -v WARNING Fourier


def periodic(offset, period):
    return lambda f: lambda x: f(((x - offset) % period) + offset)


func_period = func_period_to - func_period_from


@periodic(func_period_from, func_period)
def func_periodic(x):
    return func(x)


def func_series(n):
    @periodic(func_period_from, func_period)
    def func_series_periodic(x):
        return series(x, n)
    return func_series_periodic


class Fourier(Scene):
    def construct(self):
        self.create_axes()

        # define the original function
        graph_func = self.axes.plot(
            func_periodic,
            x_range=(func_x_range_min, func_x_range_max, plot_x_delta),
            discontinuities=func_discontinuities,
            use_smoothing=True,
            color=COLOR_FUNC
        )

        # message displayed before the formula
        txt_message = Text(message, font_size=40)
        txt_message.set_color(COLOR_MESSAGE)

        math_func = MathTex(tex_func, font_size=40)
        math_func.set_color(COLOR_FUNC)

        math_tilde = MathTex(tex_sim_or_equals, font_size=60)
        math_tilde.set_color(COLOR_TILDE)

        math_series = MathTex(tex_series, font_size=40)
        math_series.set_color(COLOR_SERIES)

        top_labels = VGroup(txt_message, math_tilde, math_series).arrange(RIGHT, buff=1)
        top_labels.to_edge(UP, buff=1)
        math_func.move_to(txt_message)

        txt_periodic = Text("periodic", font_size=22, color=COLOR_PERIODIC).next_to(math_func, DOWN, buff=0.2)

        # Animation Tutorial
        #     https://azarzadavila-manim.readthedocs.io/en/latest/animation.html
        # Animation Transform Src
        #     https://github.com/ManimCommunity/manim/blob/main/manim/animation/transform.py
        self.play(Write(txt_message), DrawBorderThenFill(self.axes))
        self.play(ReplacementTransform(txt_message, math_func), Write(txt_periodic), Create(graph_func))
        self.play(Write(math_series))
        self.play(Write(math_tilde))

        self.prepare_fourier_animation()
        self.animate_fourier_series(series_start, series_end, series_step)
        self.animate_fourier_series(series_more_start, series_more_end, series_more_step)

        self.wait()

    def prepare_fourier_animation(self):
        self.math_n_equals = MathTex(r"n = ", font_size=60, color=COLOR_N_TERM_COUNT)
        self.math_n_equals.align_to(self.axes, UR).shift(LEFT)
        self.play(FadeIn(self.math_n_equals))

        self.graph_series = None
        self.n_series = None

    def animate_fourier_series(self, start, end, step):
        each_run_time = series_run_time / (end - start)

        fade_animation_on = each_run_time >= 0.4

        # transform the graph while increasing the number of terms
        for n in range(start, end + 1, step):
            # MEMO : I could use Variable class too.
            # Variable class = ValueTracker with label
            #     https://docs.manim.community/en/stable/reference/manim.mobject.text.numbers.Variable.html

            n_series_new = Integer(n).scale(1.1).next_to(self.math_n_equals, RIGHT, buff=0.2).shift(UP * 0.03)
            n_series_new.set_color(COLOR_N_TERM_COUNT)

            # ParametricFunction
            #     https://docs.manim.community/en/stable/reference/manim.mobject.graphing.functions.ParametricFunction.html
            graph_series_new = self.axes.plot(
                func_series(n),
                x_range=[func_x_range_min, func_x_range_max, plot_x_delta],
                use_smoothing=True, color=COLOR_SERIES, stroke_width=3, stroke_opacity=0.9)
            if self.graph_series is not None:
                self.play(
                    ReplacementTransform(self.graph_series, graph_series_new, run_time=each_run_time),
                    FadeTransform(self.n_series, n_series_new, run_time=each_run_time if fade_animation_on else 0))
            else:
                self.play(Create(graph_series_new), Write(n_series_new))

            self.graph_series = graph_series_new
            self.n_series = n_series_new

    def create_axes(self):
        self.axes = (Axes(
            x_range=(x_min, x_max, x_step),
            y_range=(y_min, y_max, y_step), y_length=y_length,
            # axis_config keys available
            #   https://docs.manim.community/en/stable/reference/manim.mobject.graphing.number_line.NumberLine.html#manim.mobject.graphing.number_line.NumberLine
            axis_config={
                "include_ticks": include_ticks,
                "include_tip": include_tip
            }
        )).to_corner(DOWN, buff=1.8)

        labels = self.axes.get_axis_labels(x_label="x", y_label="").set_color(WHITE)
        self.axes.add(labels)

        if(is_pi_label):
            self.add_pi_x_labels()

    def add_pi_x_labels(self, scale=0.8, buff=0.4, buff_adjust=0.09, aligned_edge=DOWN):
        x_min = self.axes.x_range[0]
        x_max = self.axes.x_range[1]

        if show_x_0_label and x_min <= 0 and 0 <= x_max:
            self.axes.add(
                MathTex(r"0").next_to(
                    self.axes.c2p(0, 0),
                    aligned_edge, buff=buff
                ).scale(scale)
            )
        if x_min <= -PI and -PI <= x_max:
            self.axes.add(
                MathTex(r"-\pi").next_to(
                    self.axes.c2p(-PI, 0),
                    aligned_edge,
                    buff=buff + buff_adjust
                ).scale(scale)
            )
        if x_min <= PI and PI <= x_max:
            self.axes.add(
                MathTex(r"\pi").next_to(
                    self.axes.c2p(PI, 0),
                    aligned_edge,
                    buff=buff + buff_adjust
                ).scale(scale)
            )

        for n in range(-10, 10):
            if n in (-1, 0, 1):
                continue
            npi = n * PI
            if x_min <= npi and npi <= x_max:
                self.axes.add(
                    MathTex(str(n) + r"\pi").next_to(
                        self.axes.c2p(npi, 0),
                        aligned_edge,
                        buff=buff
                    ).scale(scale)
                )

Animation 15: ReplacementTransform(ParametricFunction), etc.:   0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/manim/scene/scene.py:1427: RuntimeWarning: invalid value encountered in double_scalars
  alpha = t / animation.run_time
/usr/local/lib/python3.7/dist-packages/manim/scene/scene.py:1427: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = t / animation.run_time
Animation 16: ReplacementTransform(ParametricFunction), etc.:   0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/manim/scene/scene.py:1427: RuntimeWarning: invalid value encountered in double_scalars
  alpha = t / animation.run_time
/usr/local/lib/python3.7/dist-packages/manim/scene/scene.py:1427: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = t / animation.run_time
Animation 19: ReplacementTransform(ParametricFunction), etc.:   0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/manim/scene/scene.py:1427: